In [1]:
# import statements
import pandas as pdtn_county_map
import numpy as np
import requests
#from bs4 import BeautifulSoup as bs
from IPython.core.display import HTML
import matplotlib.pyplot as plt
#import seaborn as sns
from io import StringIO
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import cartopy
import cartopy.io.shapereader as shapereader
#from cartopy.io import shapereader
%matplotlib inline
# import io
# import scipy.stats as stats
# import statsmodels.api as sm

#Note - had to # out cartopy due to error

ModuleNotFoundError: No module named 'cartopy'

In [ ]:
# display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

**Create a base map for the state of TN, broken down by county.**

In [ ]:
# Create a folium map, but probably not the best idea
# tn_map = folium.Map(location=[36,-86], zoom_start = 7)
# tn_map

In [7]:
# Create a base map by importing the base shape file for TN census map
# Annoyingly, this is broken down by Census division, not by county
# tn_census_map = gpd.read_file('../data/2018-TN-basemap/tl_2016_47_cousub.shp')
# tn_census_map.info()

In [8]:
# Check to see if COUNTYFP |is the right level to aggregate at to get county-level geometries
# tn_census_map['COUNTYFP'].nunique()

In [9]:
# Attempt to aggregate geometries by county
# tn_census_map.groupby('COUNTYFP')['geometry']

In [1]:
# Ask Michael, get this county-level base map shapefile instead
tn_county_map = gpd.read_file('../maps/tncounty.shp')
tn_county_map.plot();

In [2]:
# Take a look at the base map dataframe
tn_county_map.head()

In [3]:
# Check to make sure nothing is missing
tn_county_map.info()

In [4]:
# check the projection type
tn_county_map.crs

In [6]:
# change the projection type
tn_county_map = tn_county_map.to_crs('EPSG:4326')
print(tn_county_map.crs)
# Make sure the base map dataset is good to go
tn_county_map.head(2)

**Read in the deadly earthquakes since 1900 wikitable and turn it into a Geo DataFrame.**

In [7]:
#earthquakes = pd.read_csv('../data/earthquakes_wikitable.csv')

In [8]:
#earthquakes.head()

In [9]:
#earthquakes.info()

In [10]:
#earthquakes['origin_utc'] = pd.to_datetime(earthquakes['origin_utc'])

In [11]:
# remove excess columns
#earthquakes = earthquakes.drop(['pde_shaking_deaths', 'pde_total_deaths', 'utsu_total_deaths', 'em_dat_total_deaths', 'other_source_deaths', 'other_source_deaths_new', 'osd1', 'osd2', 'osd3'], axis = 1)

In [12]:
#earthquakes.head()

In [ ]:
# Clean up the lat/long columns
# earthquakes['lat'] = earthquakes['lat'].str.replace('?','')
# earthquakes['long'] = earthquakes['long'].str.replace('?','')

In [ ]:
# Turn the lat/long into floats
# earthquakes['long'] = pd.to_numeric(earthquakes['long'])
# earthquakes['lat'] = pd.to_numeric(earthquakes['lat'])

In [ ]:
# Create a new column named 'geometry' which combines the latitude and longitude
# earthquakes['geometry'] = earthquakes.apply(lambda x: Point((float(x.long),
#                                                             float(x.lat))),
#                                            axis = 1)
# earthquakes.head()

In [14]:
# Turn the Wikipedia Deadly Earthquakes since 1900 table into a Geo Data Frame
#deadly_earthquakes_geo = gpd.GeoDataFrame(earthquakes,
#                                          crs = tn_county_map.crs,
#                                          geometry = earthquakes['geometry'])

The Wikipedia deadly earthquakes since 1900 table is now ready for a spatial join.

**Next, pull in the USGS data for earthquakes in TN (as defined in this gist: https://gist.github.com/jakebathman/719e8416191ba14bb6e700fc2d5fccc5) and turn it into a Geo DataFrame.**

In [18]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=1900-01-01&endtime=2020-10-22&minlatitude=34.9884&maxlatitude=36.6871&minlongitude=-90.3131&maxlongitude=-81.6518'

In [19]:
r = requests.get(url)

In [20]:
usgs = pd.read_csv(StringIO(r.text))
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-10-21T21:49:45.720Z,36.070500,-89.802333,6.84,2.14,md,46.0,40.0,0.06430,0.10,...,2020-10-22T12:49:53.160Z,"2 km NNE of Cooter, Missouri",earthquake,0.17,0.49,0.092,31.0,reviewed,nm,nm
1,2020-10-21T20:28:42.930Z,36.525000,-89.569167,6.85,1.69,md,19.0,50.0,0.02596,0.03,...,2020-10-22T14:45:56.660Z,"3 km E of Marston, Missouri",earthquake,0.34,0.63,0.167,12.0,reviewed,nm,nm
2,2020-10-21T04:55:51.350Z,36.545667,-89.649000,8.01,1.61,md,24.0,44.0,0.02245,0.04,...,2020-10-21T13:04:57.100Z,"4 km NW of Marston, Missouri",earthquake,0.44,0.40,0.147,21.0,reviewed,nm,nm
3,2020-10-21T03:00:08.800Z,36.313667,-89.536167,9.50,1.23,md,17.0,65.0,0.02693,0.06,...,2020-10-21T13:18:24.430Z,"7 km NW of Ridgely, Tennessee",earthquake,0.68,0.51,0.032,10.0,reviewed,nm,nm
4,2020-10-20T18:50:21.800Z,36.267500,-89.523333,8.84,2.33,md,46.0,33.0,0.01520,0.11,...,2020-10-21T12:29:35.220Z,"3 km W of Ridgely, Tennessee",earthquake,0.15,0.30,0.142,34.0,reviewed,nm,nm


In [21]:
# Create a geometry column
usgs['geometry'] = usgs.apply(lambda x: Point((float(x.longitude),
                                            float(x.latitude))),
                              axis = 1)
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,geometry
0,2020-10-21T21:49:45.720Z,36.070500,-89.802333,6.84,2.14,md,46.0,40.0,0.06430,0.10,...,"2 km NNE of Cooter, Missouri",earthquake,0.17,0.49,0.092,31.0,reviewed,nm,nm,POINT (-89.8023333 36.0705)
1,2020-10-21T20:28:42.930Z,36.525000,-89.569167,6.85,1.69,md,19.0,50.0,0.02596,0.03,...,"3 km E of Marston, Missouri",earthquake,0.34,0.63,0.167,12.0,reviewed,nm,nm,POINT (-89.5691667 36.525)
2,2020-10-21T04:55:51.350Z,36.545667,-89.649000,8.01,1.61,md,24.0,44.0,0.02245,0.04,...,"4 km NW of Marston, Missouri",earthquake,0.44,0.40,0.147,21.0,reviewed,nm,nm,POINT (-89.649 36.5456667)
3,2020-10-21T03:00:08.800Z,36.313667,-89.536167,9.50,1.23,md,17.0,65.0,0.02693,0.06,...,"7 km NW of Ridgely, Tennessee",earthquake,0.68,0.51,0.032,10.0,reviewed,nm,nm,POINT (-89.5361667 36.3136667)
4,2020-10-20T18:50:21.800Z,36.267500,-89.523333,8.84,2.33,md,46.0,33.0,0.01520,0.11,...,"3 km W of Ridgely, Tennessee",earthquake,0.15,0.30,0.142,34.0,reviewed,nm,nm,POINT (-89.5233333 36.2675)


In [ ]:
# Subset to earthquakes that have happened within TN only, based on the place name
# Not actually necessary if we're going to plot points based on lat/long
# usgs_tn = usgs[usgs['place'].str.contains('(Tennessee)')]
# usgs_tn.info()

In [22]:
# Turn the USGS dataframe into a Geo DataFrame
usgs_geo = gpd.GeoDataFrame(usgs,
                             crs = tn_county_map.crs,
                             geometry = usgs['geometry'])

NameError: name 'tn_county_map' is not defined

In [ ]:
# Join the USGS data to the TN basemap
tn_earthquakes = gpd.sjoin(usgs_geo, tn_county_map, op = 'within')
tn_earthquakes

In [ ]:
# Since 1900, how many earthquakes per county?
tn_earthquakes['NAME'].value_counts()

**Create a map of the state of TN with all of the earthquakes that have happened since 1900.**

In [ ]:
# map the earthquakes
ax = tn_county_map.plot(figsize = (8, 10), color = 'beige')
tn_earthquakes.plot( ax = ax, column = 'NAME');
plt.show();

**Read in and clean up the census housing units by county for TN data.**

In [ ]:
tn_housing_units_by_county = pd.read_csv('../data/us_census_tn_housing_units_by_county_2010-2019.csv')[2:99]

In [ ]:
tn_housing_units_by_county.columns = tn_housing_units_by_county.iloc[0]
tn_housing_units_by_county = tn_housing_units_by_county[2:99]

In [ ]:
tn_housing_units_by_county.reset_index(drop = True)

In [ ]:
tn_housing_units_by_county.columns

In [ ]:
tn_housing_units_by_county = tn_housing_units_by_county.drop(['Estimates Base', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis = 1)

In [ ]:
tn_housing_units_by_county.columns = ['county', 'census', '2019']

In [ ]:
tn_housing_units_by_county.reset_index(drop = True)

In [ ]:
tn_housing_units_by_county['county'] = tn_housing_units_by_county['county'].str.extract(r'\.(.*) County, Tennessee')
tn_housing_units_by_county['county'] = tn_housing_units_by_county['county'].str.lower()

In [ ]:
tn_housing_units_by_county

**Read in the TN demographics data by county.**

In [ ]:
tn_demo = pd.read_csv('../data/TN-county-demographics-2010.csv')

In [ ]:
tn_demo

In [ ]:
# lower case the county to avoid merge errors
tn_demo['County'] = tn_demo['County'].str.lower()

In [ ]:
# Join the demographics and housing data into one dataframe
tn_housing_demo = tn_demo.merge(tn_housing_units_by_county, how = 'outer', left_on = 'County', right_on = 'county')
tn_housing_demo

In [ ]:
# Join in the demographic data to the TN earthquakes data
tn_earthquakes_w_demo = tn_earthquakes.merge(tn_demo, left_on = 'NAME', right_on = 'County')
# Check to make sure it is a full join
tn_earthquakes_w_demo[tn_earthquakes_w_demo['Total Population'].isnull() == True]

In [ ]:
tn_earthquakes_w_demo